This notebook computes phrases in data and replaces words with phrases. Also trains a new Word2Vec embeddings upon those phrased data.

Use either "Prepare Data 20news" part or "Prepare data fold0" part.

In [45]:
from gensim.models.phrases import Phraser, Phrases
from sklearn.datasets import fetch_20newsgroups

import gensim
import newsgroups
import pandas as pd
import numpy as np
import utility
import pickle
import DataLoader

# Prepare Data 20news


In [69]:
# This data is ready to be fed into neural network
# Can be used to create phrases aswell
trainData, trainTarget, testData, testTarget, class_dict, reverse_class_dict = newsgroups.process_newsgroups_numpy(True,False,True,False )

# Prepare data fold0(iCliniq)

In [60]:
configs = {
    "vectorSize":300,
    "trainNewModel":True,
    "dataColumn":"question",
    "maxLength":400,
    "batchSize":8,
    "embeddingType":None,
    "PreEmbed":True,
    "restore":True
}

In [9]:
dataColumn = "question"
fold0trainRaw = pd.read_csv("data/iCliniq/MainCategory-5Fold/fold 0/fold_0_train_tr.csv", encoding="utf-8")
fold0testRaw = pd.read_csv("data/iCliniq/MainCategory-5Fold/fold 0/fold_0_test_tr.csv")
fold0trainData = np.hstack((fold0trainRaw[dataColumn].values.reshape(-1,1),fold0trainRaw["category2"].values.reshape(-1,1)))
fold0testData = np.hstack((fold0testRaw[dataColumn].values.reshape(-1,1),fold0testRaw["category2"].values.reshape(-1,1)))
fold0ClassDict = DataLoader.DataHandler.getUniqueClassMapDict(fold0testData[:,1])

trainData = fold0trainData
trainData, trainTarget, _ = DataLoader.DataHandler.masterPreprocessor(trainData,shuffle=True,classDict=fold0ClassDict,maxLength=configs["maxLength"])
testData, testTarget, _ = DataLoader.DataHandler.masterPreprocessor(fold0testData,shuffle=True,classDict=fold0ClassDict,maxLength=configs["maxLength"])

Outputs converted to numerical forms
Input text claned
Input text split into tokens and all inputs padded to maximum length
Outputs converted to numerical forms
Input text claned
Input text split into tokens and all inputs padded to maximum length


# Get Phrases (BiGram and TriGram)

In [70]:
all_data = np.array(list(trainData)+list(testData))

In [71]:
testData = all_data
# Convert test data to bigram-trigram
test_bigram_model = Phraser(Phrases(testData, min_count=10, threshold=100))
test_bigram = [test_bigram_model[line] for line in testData]
test_trigram_model = Phraser(Phrases(test_bigram, min_count=10, threshold=100))
test_trigram = [test_trigram_model[line] for line in test_bigram]

In [72]:
# Convert train data to bigram-trigram
train_bigram_model = Phraser(Phrases(trainData, min_count=10, threshold=100))
# change data with phrases
train_bigram = [train_bigram_model[line] for line in trainData]


train_trigram_model = Phraser(Phrases(train_bigram, min_count=10, threshold=100))
# change data with phrases
train_trigram = [train_trigram_model[line] for line in train_bigram]

In [73]:
train_trigram = np.array(train_trigram)
test_trigram = np.array(test_trigram)

In [ ]:
# train_data_trigram = np.vstack((train_trigram, trainTarget)).T
# test_data_trigram = np.vstack((test_trigram, testTarget)).T

In [74]:
# if you do not want to save just change it to false
should_save = True
if should_save:
    pd.DataFrame(train_trigram).to_csv("data//phrased//train_trigram.csv")
    pd.DataFrame(test_trigram).to_csv("data//phrased//test_trigram.csv")

In [75]:
phrases = []
for i in range(len(testData)):
    for phrase in test_trigram_model[test_bigram[i]]:
        if phrase is not "[None]" and "_" in phrase and phrase not in phrases:
            phrases.append(phrase)
print(len(phrases))

2137


In [77]:
with open("data//phrased//phrases_icliniq.txt", "w") as f:
    for item in phrases:
        f.write("%s\n" % item)

In [25]:
# Change encoding (there may be error otherwise)
for i in range(len(test_trigram)):
    test_trigram[i] = [a.encode("utf-8") for a in test_trigram[i]]

# Train Word2Vec with phrased data and save it to hard drive

In [78]:
word_vectors = w2v_model.wv

NameError: name 'w2v_model' is not defined

In [79]:
all_data = [list(a) for a in all_data] 
data = all_data
w2v_model = gensim.models.Word2Vec(data, size=300, window=10, min_count=2,workers=10)
w2v_model.train(data, total_examples=len(data), epochs=10)

(15979656, 24122880)

In [80]:
w2v_model.wv.save("Embeddings//phrased_embedding.w2v")